# Task 2g Solution

In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import pickle

In [2]:
detection_sci = np.load('data/detection_sci.npy') # read detection sci image
detection_wht = np.load('data/detection_wht.npy') # read detection wht image

--- cut out a portion of the image for analysis

In [3]:
x = 3100 # x-center of the image
y = 1800 # y-center of the image
r = 200 # width/2 of cutout, must be int

In [4]:
detection_sci = detection_sci[x-r:x+r, y-r:y+r] # cutout a portion of the science image
detection_wht = detection_wht[x-r:x+r, y-r:y+r] # cutout a portion of the weight image

--- define the noise in each pixel and make a significance map (signal/noise)

In [5]:
detection_noise = 1./np.sqrt(detection_wht) #Â conversion from weight to noise
detection_sig = detection_sci/detection_noise # signifance map

--- now run segmentation on the image

In [6]:
from photutils import detect_sources

In [7]:
threshold = 2.5
npixels = 5

In [8]:
segm = detect_sources(detection_sig, threshold, npixels)

In [9]:
from photutils import SourceCatalog, CircularAperture, aperture_photometry

In [10]:
cat = SourceCatalog(detection_sci, segm)

--- get a list of positions (x,y) of the sources

In [11]:
positions = []
for obj in cat:
    positions.append(np.transpose((obj.xcentroid, obj.ycentroid)))

--- make a CicrcularAperture object. This can be plotted but is mostly used for the aperture photometry.

In [12]:
aper_r = 5. # radius of aperture in pixels
apertures = CircularAperture(positions, aper_r)

--- now open all the individual filters

In [13]:
image_dir = 'data'

In [14]:
filters = ['f435w','f606w', 'f775w','f850lp', 'f105w','f125w','f140w','f160w']

In [15]:
aper_cat = {}

In [16]:
for f in filters:

    # --- read in the science image and slice for our region
    sci = fits.getdata(f'{image_dir}/{f}_sci.fits')[x-r:x+r, y-r:y+r]
    # --- run aperture photometry to get photometry table
    phot_table = aperture_photometry(sci, apertures)
    # --- convert aperture flux column to an array and place in output dictionary
    aper_cat[f+'_signal'] = np.array(phot_table['aperture_sum'])

    # --- read in the weight image and slice for our region
    wht = fits.getdata(f'{image_dir}/{f}_wht.fits')[x-r:x+r, y-r:y+r]
    # --- define the noise image
    noise = 1/np.sqrt(wht)
    # --- square this ready for summing
    noisesq = noise**2
    # --- run aperture photometry to get photometry table
    phot_table = aperture_photometry(noisesq, apertures)
    # --- convert aperture flux column to an array and place in output dictionary
    aper_cat[f+'_noise'] = np.sqrt(np.array(phot_table['aperture_sum']))
    print(phot_table)

 id      xcenter            ycenter            aperture_sum     
           pix                pix                               
--- ------------------ ------------------ ----------------------
  1  34.04484025039375  6.418786122833452 2.9655886767036123e-05
  2  87.58659874116391 1.2611732175528336 2.2176292984873066e-05
  3 214.97369165809405 1.3751704516568082 2.1901527939697553e-05
  4 254.31752262285698 3.7691785793516055   2.99743823057748e-05
  5 140.25753358191847 14.941802884480326 2.9953427187580018e-05
  6  389.4668984021568 17.470532448997684   3.15668375490638e-05
  7  54.19762034581787  33.63096552126966  3.116754283997316e-05
  8  141.7174763291304 47.374144820287285 3.0222330532094282e-05
  9 181.29488673816928 39.113288788324766 3.0029167933798825e-05
 10  311.1466971563536 36.990127258537164  3.052770430401386e-05
...                ...                ...                    ...
114  96.76100420253387  363.0159309884573 3.0326220603842604e-05
115  339.6598568120095 37

 id      xcenter            ycenter            aperture_sum     
           pix                pix                               
--- ------------------ ------------------ ----------------------
  1  34.04484025039375  6.418786122833452 0.00026264949015693994
  2  87.58659874116391 1.2611732175528336  0.0001472870299973278
  3 214.97369165809405 1.3751704516568082 0.00016174366294716884
  4 254.31752262285698 3.7691785793516055 0.00021366136201654432
  5 140.25753358191847 14.941802884480326 0.00020072564407105025
  6  389.4668984021568 17.470532448997684 0.00020029636947525856
  7  54.19762034581787  33.63096552126966 0.00020311269131424148
  8  141.7174763291304 47.374144820287285  0.0001999809739620297
  9 181.29488673816928 39.113288788324766 0.00021186773013230344
 10  311.1466971563536 36.990127258537164 0.00022315071153187205
...                ...                ...                    ...
114  96.76100420253387  363.0159309884573 0.00020341655886983723
115  339.6598568120095 37

In [17]:
pickle.dump(aper_cat, open('data/aper_cat.p','wb'))